# Setup: Installing Required Libraries

Before we begin, we need to install the necessary Python libraries. Run the cell below to install all dependencies for this notebook.

In [ ]:
# Install required libraries - CONSISTENT VERSIONS
# Both notebooks use: chromadb 0.5.3 + openai 0.28.1
!pip install -q chromadb==0.5.3 openai==0.28.1 sentence-transformers==3.0.1 transformers torch

print("✅ All libraries installed successfully!")
print("⚠️  IMPORTANT: Please restart your kernel/runtime now before running the next cell!")

### 💡 Installation Note

If you see **dependency conflict warnings** during installation, you can safely **ignore them** - they won't affect this notebook.

**Remember:** Always restart your runtime after installation! (Runtime → Restart runtime)

---

In [ ]:
# Upload and extract the database from Notebook 1
import zipfile
import os

try:
    from google.colab import files
    print("📤 Please upload the chromadb_database.zip file...")
    uploaded = files.upload()
    
    if 'chromadb_database.zip' in uploaded:
        # Extract the zip file
        with zipfile.ZipFile('chromadb_database.zip', 'r') as zip_ref:
            zip_ref.extractall('./uploaded_db')
        print("✅ Database extracted successfully!")
    else:
        print("❌ chromadb_database.zip not found. Please upload it.")
        
except ImportError:
    # Not in Colab - check if zip exists locally
    if os.path.exists('chromadb_database.zip'):
        with zipfile.ZipFile('chromadb_database.zip', 'r') as zip_ref:
            zip_ref.extractall('./uploaded_db')
        print("✅ Database extracted from local zip file!")
    else:
        print("❌ chromadb_database.zip not found. Please place it in the current directory.")

## Verify Database Connection

Now let's verify that the collection from Notebook 1 was loaded successfully.

In [ ]:
# Importing
import os
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
# Reconnecting to the persistent DB
chroma_client = chromadb.PersistentClient(path="./uploaded_db")

In [ ]:
# Reopening the collection and verifying it has data
# This cell will check if the collection exists and contains the expected documents from notebook 1

try:
    local_collection = chroma_client.get_collection("my_documents_locally")
    count = local_collection.count()
    print(f"✅ Successfully connected to collection 'my_documents_locally'")
    print(f"   Documents in collection: {count}")

    if count == 0:
        print("\n⚠️  WARNING: Collection exists but contains no documents!")
        print("   Please run the first notebook (1.Creating_Embeddings_using_Chroma.ipynb) completely.")
    elif count < 6:
        print(f"\n⚠️  WARNING: Expected 6 documents but found {count}")
        print("   Please re-run the first notebook to ensure all documents are added.")

except ValueError as e:
    print("❌ ERROR: Collection 'my_documents_locally' does not exist!")
    print("\n📝 SOLUTION:")
    print("   1. Go back to the first notebook: '1.Creating_Embeddings_using_Chroma.ipynb'")
    print("   2. Run ALL cells in that notebook to create and populate the collection")
    print("   3. Then return to this notebook")
    print(f"\nTechnical details: {e}")
    raise

Now we can run a semantic search using `query()` function. When we call it, Chroma takes 2 steps:
1. It **embeds our query text** using the same embedding function that was used for the documents in the collection.
2. It **compares the query embedding with all stored embeddings** and then returns the most relevant results.

In this example, we tell Chroma to return the top 2 most relevant documents:

In [ ]:
query_text = "What are the foundational principles and technologies used to secure modern internet traffic?"

results = local_collection.query(
    query_texts = [query_text],
    n_results = 2,
)

To make the output easier to read, let’s loop through the results and print a short preview of each one. The output includes **the matching documents, their IDs and the distance scores** which is a measure of similarity (the smaller the distance, the closer the match).

In [ ]:
# Displaying results in readable format
for rank, (document, document_id, distance_score) in enumerate(
    zip(results["documents"][0], results["ids"][0], results["distances"][0]),
    start=1):
    preview = document[:600]
    print(f"Result {rank}")
    print(f"• ID: {document_id}")
    print(f"• Distance: {distance_score:.4f}")
    print(f"• Preview: {preview}…")
    print("-" * 80)

## 1.1 Filtering Semantic Search with Metadata

Semantic search is powerful, but sometimes you want to **combine semantic similarity with specific filters**. For example, you might want to find documents about "encryption" but only from the "cryptography" category, or only beginner-level documents.

ChromaDB allows you to add a `where` clause to your queries, just like you learned in the first notebook. This combines the best of both worlds:
- **Semantic search** finds conceptually relevant documents
- **Metadata filters** narrow down results to specific criteria

Let's see this in action.

### Example 1: Search with Category Filter

Let's search for documents about "network protection" but only from the "architecture" category:

In [ ]:
query_text = "How to protect networks from attackers?"

# Semantic search with category filter
results = local_collection.query(
    query_texts=[query_text],
    n_results=2,
    where={"category": "architecture"},  # Only architecture documents
    include=["documents", "metadatas", "distances"]
)

print(f"Query: {query_text}")
print(f"Filter: category = 'architecture'\n")

for rank, (doc, doc_id, metadata, distance) in enumerate(
    zip(results["documents"][0], results["ids"][0], results["metadatas"][0], results["distances"][0]),
    start=1):
    print(f"Result {rank}:")
    print(f"  ID: {doc_id}")
    print(f"  Category: {metadata['category']}")
    print(f"  Distance: {distance:.4f}")
    print(f"  Preview: {doc[:200]}...\n")

### Example 2: Search with Difficulty Filter

Now let's find beginner-friendly documents about authentication:

In [ ]:
query_text = "How do authentication systems work?"

# Semantic search with difficulty filter
results = local_collection.query(
    query_texts=[query_text],
    n_results=3,
    where={"difficulty": "beginner"},  # Only beginner-level documents
    include=["metadatas", "distances"]
)

print(f"Query: {query_text}")
print(f"Filter: difficulty = 'beginner'\n")

if results["ids"][0]:
    for rank, (doc_id, metadata, distance) in enumerate(
        zip(results["ids"][0], results["metadatas"][0], results["distances"][0]),
        start=1):
        print(f"Result {rank}: {doc_id}")
        print(f"  Category: {metadata['category']}, Difficulty: {metadata['difficulty']}")
        print(f"  Distance: {distance:.4f}\n")
else:
    print("No beginner-level documents found matching this query.")

### Example 3: Complex Filters with Operators

You can also use operators like `$gte` (greater than or equal), `$in` (in list), etc.:

In [ ]:
query_text = "Modern security approaches"

# Find documents from 2010 or later
results = local_collection.query(
    query_texts=[query_text],
    n_results=3,
    where={"year": {"$gte": 2010}},  # Documents from 2010 onwards
    include=["metadatas", "distances"]
)

print(f"Query: {query_text}")
print(f"Filter: year >= 2010\n")

for rank, (doc_id, metadata, distance) in enumerate(
    zip(results["ids"][0], results["metadatas"][0], results["distances"][0]),
    start=1):
    print(f"Result {rank}: {doc_id}")
    print(f"  Category: {metadata['category']}, Year: {metadata['year']}")
    print(f"  Distance: {distance:.4f}\n")

**Key Takeaway:** Filtered semantic search is incredibly powerful. It lets you:
- Find semantically relevant content ("what does it mean?")
- While applying specific business logic ("show me only X type of documents")

This is essential for building production RAG systems where you need to control which documents can be retrieved based on user permissions, document types, dates, or other criteria.

### 📝 EXERCISE 1: Try Your Own Semantic Search (5-7 minutes)

**What you'll practice:** Running semantic searches and understanding how query phrasing affects results.

**Your task:**
1. Create your own query about a cybersecurity topic (e.g., "How do hackers steal passwords?", "What is modern network security?", "How do viruses work?")
2. Use `local_collection.query()` to search for the top 3 most relevant documents
3. Print the results showing the document IDs and distance scores
4. Experiment: Try rephrasing your question differently - do you get similar results?

**Hint:** Use the same structure as the example above. Remember to set `n_results=3` to get 3 documents.

**Expected outcome:** You should see that semantically similar questions return similar documents, even if worded differently. Lower distance scores mean better matches.

In [ ]:
# YOUR CODE HERE


## 1.2 Optimizing n_results: How Many Documents Should You Retrieve?

When performing semantic search for RAG, one critical parameter is `n_results` - how many documents to retrieve. This isn't just a technical detail; it's a key design decision that affects your system's quality, cost, and performance.

### The Trade-offs

**Too Few Documents (n_results = 1-2):**
- ✅ **Pros:**
  - Faster retrieval
  - Lower LLM cost (less context to process)
  - More focused answers
- ❌ **Cons:**
  - **Low recall**: Might miss relevant information spread across multiple documents
  - Risk of incomplete answers
  - Single point of failure if top result isn't perfect

**Too Many Documents (n_results = 10+):**
- ✅ **Pros:**
  - **High recall**: More likely to capture all relevant information
  - Better coverage of the topic
- ❌ **Cons:**
  - **Context size explosion**: LLMs have token limits (e.g., GPT-4: 8k-128k tokens)
  - Higher costs (more tokens = more $$$)
  - **Noise**: Irrelevant documents can confuse the LLM
  - Slower processing
  - LLM may struggle to synthesize information from too many sources

### Finding the Sweet Spot

Let's experiment with different values of `n_results` to see how it affects our results:

In [ ]:
query_text = "What are the main security threats?"

print(f"Query: {query_text}\n")
print("=" * 80)

# Test different n_results values
for n in [1, 3, 5]:
    results = local_collection.query(
        query_texts=[query_text],
        n_results=n,
        include=["distances", "documents"]
    )

    print(f"\nn_results = {n}:")
    print(f"  Document IDs: {results['ids'][0]}")
    print(f"  Distance scores: {[f'{d:.4f}' for d in results['distances'][0]]}")

    # Calculate approximate token count (rough estimate: 1 token ≈ 4 characters)
    total_chars = sum(len(doc) for doc in results["documents"][0])
    approx_tokens = total_chars // 4
    print(f"  Approx context tokens: ~{approx_tokens}")

print("\n" + "=" * 80)

### Practical Guidelines

**1. Start with n_results = 3-5** for most use cases:
   - Good balance between recall and context size
   - Works well with most LLMs' context windows
   - Manageable cost

**2. Adjust based on your documents:**
   - **Short documents** (tweets, Q&A pairs): Can use higher n_results (10-20)
   - **Long documents** (articles, papers): Use lower n_results (2-5)
   - **Chunked documents**: Consider 5-10 chunks

**3. Consider your LLM's context window:**
   - GPT-3.5-turbo: 4k tokens → Keep context under 3k
   - GPT-4: 8k-32k tokens → More flexibility
   - Local models: Often 2k-4k → Be conservative

**4. Monitor distance scores:**
   - If the 3rd result has a distance > 0.5, the remaining results probably won't help
   - Use a **distance threshold** instead of fixed n_results

**5. Advanced technique - Reranking:**
   - Retrieve more documents (n=10-20)
   - Use a reranking model to select the best 3-5
   - Send only the reranked results to the LLM

### Example: Dynamic n_results with Distance Threshold

In [ ]:
def smart_retrieve(collection, query, max_results=5, distance_threshold=0.6):
    """
    Retrieve documents but stop if distance scores get too high (low similarity)
    """
    results = collection.query(
        query_texts=[query],
        n_results=max_results,
        include=["documents", "distances"]
    )

    # Filter out documents with distance > threshold
    filtered_docs = []
    filtered_ids = []

    for doc, doc_id, dist in zip(
        results["documents"][0],
        results["ids"][0],
        results["distances"][0]
    ):
        if dist <= distance_threshold:
            filtered_docs.append(doc)
            filtered_ids.append(doc_id)
        else:
            print(f"  Skipping {doc_id} (distance {dist:.4f} > threshold {distance_threshold})")

    return filtered_docs, filtered_ids

# Test it
query = "How does public key cryptography work?"
print(f"Query: {query}\n")

docs, ids = smart_retrieve(local_collection, query, max_results=5, distance_threshold=0.5)
print(f"\nRetrieved {len(docs)} high-quality documents: {ids}")

**Key Takeaway:**

Don't just blindly set `n_results=5`. Think about:
- Your document length and structure
- Your LLM's context window
- Your quality vs. cost trade-offs
- The semantic distance of retrieved results

For most applications, **n_results=3** with a **distance threshold of 0.5-0.6** is a good starting point. Then tune based on your specific needs!

# 2. RAG (Retrieval-Augmented-Generation)

We’ll keep doing the same search, but now we’ll hand the retrieved text to an LLM and have it compose an answer while staying grounded in our documents.

## 2.1 OpenAI API

We’ll access LLM through the OpenAI API. To do this, we first load the API key from the environment and then create a client object that we will use to send requests to the model.

In [ ]:
import os
from getpass import getpass

# Configure OpenAI API key
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("\n✅ OpenAI API configured!")

# Set the model to use
OPENAI_MODEL = "gpt-3.5-turbo"
print(f"🤖 Using model: {OPENAI_MODEL}")

In [ ]:
import openai


In [ ]:
# Configure OpenAI for old API (0.28.1)
openai.api_key = OPENAI_API_KEY

We will query Chroma collection to get the most relevant passages for the question. Here we ask for the top 3 results, and we also include their IDs. These IDs help us later show where the information came from. If no results are found, we stop early and tell the user that nothing matched.

Finally, we'll send retrieved passages along with the question to the OpenAI model `gpt-3.5-turbo`. The `system` message is an instruction that controls how the model should behave. The `user` message then supplies both the sources and the question, so the model has all the context it needs.

### 📝 EXERCISE 2: Build Your Own RAG Query with OpenAI (10-12 minutes)

**What you'll practice:** Creating a complete RAG pipeline from retrieval to answer generation.

**Your task:**
1. Think of a new question about the cybersecurity documents (e.g., "What is Stuxnet?", "How does Zero Trust work?", "What makes phishing attacks successful?")
2. Retrieve the top 2 relevant documents from `local_collection`
3. Create a context block from the retrieved documents
4. Send the context and your question to OpenAI's API to generate an answer
5. Print the final answer

**Hint:** Follow the same pattern as the example above. You'll need to:
- Use `local_collection.query()` to retrieve documents
- Format the context with document IDs
- Use `openai.ChatCompletion.create()` with messages in the format: `[{"role": "system", "content": "..."}, {"role": "user", "content": "..."}]`

**Expected outcome:** You should get a clear, cited answer that references specific source documents.

In [ ]:
# Step 3: Generating an answer using the retrieved context
response = openai.ChatCompletion.create(
    model=OPENAI_MODEL,
    messages=[
        {
            "role": "system",
            "content": (
                "You are a precise assistant that must answer ONLY using the provided sources. "
                "If the answer is not fully supported by the sources, explain that the information is not in the provided material and suggest how the user might rephrase their question. "
                "At the end, cite sources by their bracket labels, e.g., [1], [2]."
            )
        },
        {
            "role": "user",
            "content": f"SOURCES:\n{context}\n\nQUESTION: {query_text}\n\nANSWER:"
        }
    ],
    temperature=0
)

print(response['choices'][0]['message']['content'])

Finally, we'll send retrieved passages along with the question to the OpenAI model `gpt-5-nano`. The `system` message is an instruction that controls how the model should behave. The `user` message then supplies both the sources and the question, so the model has all the context it needs.

### 📝 EXERCISE 2: Build Your Own RAG Query with OpenAI (10-12 minutes)

**What you'll practice:** Creating a complete RAG pipeline from retrieval to answer generation.

**Your task:**
1. Think of a new question about the cybersecurity documents (e.g., "What is Stuxnet?", "How does Zero Trust work?", "What makes phishing attacks successful?")
2. Retrieve the top 2 relevant documents from `local_collection`
3. Create a context block from the retrieved documents
4. Send the context and your question to OpenAI's API to generate an answer
5. Print the final answer

**Hint:** Follow the same pattern as the example above. You'll need to:
- Use `local_collection.query()` to retrieve documents
- Format the context with document IDs
- Use `client.chat.completions.create()` to generate the answer

**Expected outcome:** You should get a clear, cited answer that references specific source documents.

## 2.1.1 RAG Evaluation and Source Attribution

So far, we've built a RAG pipeline that retrieves documents and generates answers. But there's a critical question we haven't addressed: **How do we know if the LLM actually used the retrieved sources, or if it just made up an answer?**

This is one of the biggest challenges in production RAG systems:

1. **No verification that sources are actually used**: After getting an LLM answer, there's often no check whether the answer actually came from the retrieved context
2. **Hallucinations**: LLMs can confidently generate information that wasn't in the sources at all
3. **Source attribution**: Even when the answer is correct, it's hard to verify which specific source documents were actually used

Let's explore techniques to address these issues.

### Technique 1: Explicit Citation Requirements

The simplest approach is to **force the LLM to cite its sources** in the system prompt. Let's compare answers with and without citation requirements:

In [ ]:
# Example WITHOUT citation requirement
query_text = "What is quantum computing?"

response = local_collection.query(
    query_texts=[query_text],
    n_results=2,
    include=["documents"]
)

docs = response["documents"][0]
ids = response["ids"][0]

if docs:
    context_blocks = [f"[{i+1} | {ids[i]}]\n{docs[i]}" for i in range(len(docs))]
    context = "\n\n".join(context_blocks)
    
    # Weak system prompt - no citation requirement
    resp_no_citation = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Answer the question using the provided sources."
            },
            {
                "role": "user",
                "content": f"SOURCES:\n{context}\n\nQUESTION: {query_text}"
            }
        ]
    )
    
    print("❌ WITHOUT Citation Requirement:")
    print(resp_no_citation['choices'][0]['message']['content'])
    print("\n" + "="*80 + "\n")

In [ ]:
    # Now WITH strong citation requirement
    resp_with_citation = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a precise assistant. Answer ONLY using the provided sources. "
                    "You MUST cite every claim with [1], [2], etc. "
                    "If the answer is not in the sources, say 'The provided sources do not contain this information.'"
                )
            },
            {
                "role": "user",
                "content": f"SOURCES:\n{context}\n\nQUESTION: {query_text}"
            }
        ]
    )
    
    print("✅ WITH Citation Requirement:")
    print(resp_with_citation['choices'][0]['message']['content'])

**Key Observation:** Without explicit citation requirements, the LLM may provide an answer that sounds plausible but isn't grounded in the sources. With citations, we can trace every claim back to a specific document.

### Technique 2: Programmatic Source Verification

A more robust approach is to **automatically verify** if the LLM's answer actually uses content from the retrieved sources. We can do this by checking for text overlap:

In [ ]:
def verify_source_usage(answer, source_docs, min_overlap_words=5):
    """
    Check if the LLM answer contains content from the source documents.
    Returns a dict with verification metrics.
    """
    answer_lower = answer.lower()
    answer_words = set(answer_lower.split())
    
    verification = {
        "uses_sources": False,
        "source_overlaps": [],
        "total_overlap_words": 0
    }
    
    for i, doc in enumerate(source_docs):
        doc_lower = doc.lower()
        doc_words = set(doc_lower.split())
        
        # Find common words (excluding very short words)
        overlap = answer_words & doc_words
        meaningful_overlap = {w for w in overlap if len(w) > 3}
        
        if len(meaningful_overlap) >= min_overlap_words:
            verification["uses_sources"] = True
            verification["source_overlaps"].append({
                "source_index": i + 1,
                "overlap_count": len(meaningful_overlap),
                "sample_words": list(meaningful_overlap)[:10]
            })
            verification["total_overlap_words"] += len(meaningful_overlap)
    
    return verification

# Test it with a real RAG query
test_query = "What are encryption algorithms?"
response = local_collection.query(
    query_texts=[test_query],
    n_results=2,
    include=["documents"]
)

test_docs = response["documents"][0]
test_ids = response["ids"][0]

if test_docs:
    context_blocks = [f"[{i+1} | {test_ids[i]}]\n{test_docs[i]}" for i in range(len(test_docs))]
    context = "\n\n".join(context_blocks)
    
    # Get LLM answer using ChatCompletion API
    test_resp = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=[
            {
                "role": "system",
                "content": "Answer using the provided sources."
            },
            {
                "role": "user",
                "content": f"SOURCES:\n{context}\n\nQUESTION: {test_query}"
            }
        ]
    )
    
    answer = test_resp['choices'][0]['message']['content']
    
    # Verify source usage
    verification = verify_source_usage(answer, test_docs)
    
    print(f"Query: {test_query}\n")
    print(f"Answer: {answer}\n")
    print("="*80)
    print(f"\n✓ Verification Results:")
    print(f"  Uses sources: {verification['uses_sources']}")
    print(f"  Total word overlap: {verification['total_overlap_words']}")
    for overlap_info in verification["source_overlaps"]:
        print(f"  Source [{overlap_info['source_index']}]: {overlap_info['overlap_count']} overlapping words")

**Key Takeaway - RAG Evaluation:**

In production RAG systems, you should:

1. **Enforce citations** in system prompts - make the LLM cite sources with [1], [2], etc.
2. **Verify programmatically** - use overlap analysis or semantic similarity to check if answers use the sources
3. **Monitor hallucinations** - if word overlap is too low, the LLM may be inventing information
4. **Build feedback loops** - log cases where verification fails for continuous improvement

These techniques help ensure your RAG system stays grounded in facts and doesn't hallucinate information that isn't in your knowledge base.

## 2.2 Local Model from Hugging Face

In the previous section, we combined local retrieval with ChromaDB (embeddings stored on disk) and then sent the retrieved passages to OpenAI’s API for generation. That worked well but it also meant our data had to leave our computer and be processed by an external service.

What if we don’t want to send private documents outside our environment? In this section we’ll keep **everything on-device by running a model locally**. We'll use a pretrained instruction-tuned model from HuggingFace called `Qwen/Qwen2.5-1.5B-Instruct`.

This model is small enough to run on a laptop or a single GPU, yet powerful enough to follow instructions and generate answers. With it, both retrieval and generation happen locally:
- our data never leave our computer
- we avoid per-request API costs
- we reduce reliance on external services and avoid latency from network calls

You can read more about this model on [HuggingFace website](https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct).

We start by importing the Hugging Face `transformers` library, which gives us the tools to load and run local language models.

In [ ]:
# Importing
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

We then load two key components:
- **Tokenizer**: This converts our input text (the context and the question) into numerical tokens the model can understand and process. Later, the tokenizer also converts the model’s output tokens back into human-readable text.
- **Model**: This is the actual neural network with all its trained weights.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pulling down the model weights and config
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",       # place the model on GPU if available, CPU otherwise
    torch_dtype = "float16")

Now we'll create custom function tht ties everything together. First, it opens the Chroma collection and retrieves the top-k most relevant documents for the given question. Those documents are combined into a prompt which instructs the model to answer only using the retrieved context. The prompt is then tokenized and passed to the language model for generation. The model produces new tokens, which are decoded back into text to form the final answer.

In [ ]:
def get_answer(client, collection_name: str, question: str, k: int = 2, max_new_tokens: int = 120):
    # Opening the collection
    col = client.get_collection(collection_name)

    # Retrieving top-k documents
    res = col.query(query_texts=[question], n_results=k, include=["documents"])
    docs = res["documents"][0] if res.get("documents") else []
    if not docs:
        return "Insufficient context."

    # The prompt
    context = "\n\n".join(docs)
    prompt = (
        "You are a precise assistant.\n"
        "Answer ONLY using the CONTEXT below."
        "If the CONTEXT is insufficient, reply exactly: Insufficient context.\n"
        "Keep the answer short (1–2 sentences).\n\n"
        "----- CONTEXT -----\n"
        f"{context}\n"
        "-------------------\n"
        f"QUESTION: {question}\n"
        "Answer:"
    )

    # Tokenizing and generating
    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)
    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    # Decoding only the newly generated tokens
    answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return answer.strip()

In [ ]:
question = "What is the difference between symmetric and asymmetric encryption?"

# Calling the RAG function
answer = get_answer(
    client = chroma_client,
    collection_name = "my_documents_locally",
    question = question,
    k = 2
)

# Displaying the output
print("Question:", question)
print("Answer:", answer)